In [1]:
import sys
sys.path.append('../src')
sys.path.append('../scripts')

import warnings
warnings.filterwarnings('ignore')

In [2]:
from jobDescriptionScraper import LinkedInDescriptionScraper
from jobListingScraper import LinkedInJobListingScraper

In [3]:
from resumeRestructre import save_restructured_resume

In [4]:
# Restructuring the input resume
restructured_resume_path, restructured_resume_json= save_restructured_resume(r"C:\Users\mnsnn\Downloads\Nishan 2024 March Resume.pdf", 1)

In [5]:
%%time
# Job Listing Scraping
job_listing_scraper = LinkedInJobListingScraper()
username = 'regenlinkedin.yfn84@aleeas.com'
password = 'Nishan@123'
jobs = job_listing_scraper.scrape_linkedin_jobs("Machine Learning Engineer", "India", 5, username, password)

Scraping page starting at 0


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".scaffold-layout__list-container"}
  (Session info: MicrosoftEdge=128.0.2739.79); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF62A3C3A22+10594]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF62A360449+323337]
	(No symbol) [0x00007FF62A17F16E]
	(No symbol) [0x00007FF62A1C656E]
	(No symbol) [0x00007FF62A1C65F5]
	(No symbol) [0x00007FF62A204157]
	(No symbol) [0x00007FF62A1E7BBF]
	(No symbol) [0x00007FF62A1BBEDD]
	(No symbol) [0x00007FF62A201B07]
	(No symbol) [0x00007FF62A1E77E3]
	(No symbol) [0x00007FF62A1BB3EC]
	(No symbol) [0x00007FF62A1BA8CD]
	(No symbol) [0x00007FF62A1BAFB1]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF62A54E4C4+918756]
	Microsoft::Applications::Events::EventProperty::~Even

In [7]:
%%time
# Job Description Scraping
job_description_scraper = LinkedInDescriptionScraper()
jobs['Job Descriptions'] = jobs['Job ID'].apply(lambda x : job_description_scraper.getDescription(job_id=x))

CPU times: total: 15.6 ms
Wall time: 11.8 s


In [8]:
from pdfLoader import loadPdfContent
old_resume_content = loadPdfContent(restructured_resume_path)

In [17]:
# Old Resume Scoring
from resumeRelevancyScore import RelevanceScorer
resume_relevence_scorer = RelevanceScorer()

jobs['Old Resume Score'] = jobs['Job Descriptions'].apply(lambda x : resume_relevence_scorer.calculate_relevance_score(old_resume_content, x))

In [18]:
jobs

,Job Title,Job Post Url,Company Name,Location,Job ID,Job Descriptions,Old Resume Score,ReGen JSON,ReGen FilePath,New Resume Score
0,Generative AI Engineer,https://www.linkedin.com/jobs/view/4009701610/...,Deloitte,"Bengaluru, Karnataka, India (Hybrid)",4009701610,"Hi, We are looking for Candidates who are havi...",82.75,"{'name': 'Muhammed Nishan', 'contact': {'email...",../generatedResumes\4009701610.pdf,83.38
1,Machine Learning Engineer,https://www.linkedin.com/jobs/view/3992115810/...,KeyValue Software Systems,"Kochi, Kerala, India (On-site)",3992115810,About KeyValue\n\nKeyValue is a product develo...,63.32,"{'name': 'Muhammed Nishan', 'contact': {'email...",../generatedResumes\3992115810.pdf,66.15
2,Specialist - Machine Learning Engineer,https://www.linkedin.com/jobs/view/3992430870/...,LTIMindtree,"Bengaluru, Karnataka, India (Hybrid)",3992430870,Role Description\n\n3 – 6 years of relevant wo...,80.60,"{'name': 'Muhammed Nishan', 'contact': {'email...",../generatedResumes\3992430870.pdf,77.35
3,ML Engineer II,https://www.linkedin.com/jobs/view/4023099196/...,Nissan Motor Corporation,"Trivandrum, Kerala, India",4023099196,Nissan is a pioneer in Innovation and Technolo...,62.56,"{'name': 'Muhammed Nishan', 'contact': {'email...",../generatedResumes\4023099196.pdf,65.34
4,Machine Learning Engineer,https://www.linkedin.com/jobs/view/4016615439/...,VerSe Innovation,"Bengaluru, Karnataka, India (On-site)",4016615439,About the team\n\nAre you passionate about mac...,73.71,"{'name': 'Muhammed Nishan', 'contact': {'email...",../generatedResumes\4016615439.pdf,78.26


In [14]:
from reGenerate import regen_resume
import pandas as pd

def apply_regen_resume(row):
    regen_json, output_filepath = regen_resume(row['Job Descriptions'], restructured_resume_json, row['Job ID'])
    return pd.Series({'ReGen JSON': regen_json, 'ReGen FilePath': output_filepath})

jobs[['ReGen JSON', 'ReGen FilePath']] = jobs.apply(apply_regen_resume, axis=1)

In [15]:
from resumeRelevancyScore import RelevanceScorer
resume_relevence_scorer = RelevanceScorer()

# Calculating the relavancy score of new resume
jobs['New Resume Score'] = jobs[['Job Descriptions', 'Job ID']].apply(lambda x : resume_relevence_scorer.calculate_relevance_score_new(x[1], x[0]), axis=1)

In [16]:
jobs

,Job Title,Job Post Url,Company Name,Location,Job ID,Job Descriptions,Old Resume Score,ReGen JSON,ReGen FilePath,New Resume Score
0,Generative AI Engineer,https://www.linkedin.com/jobs/view/4009701610/...,Deloitte,"Bengaluru, Karnataka, India (Hybrid)",4009701610,"Hi, We are looking for Candidates who are havi...",82.75,"{'name': 'Muhammed Nishan', 'contact': {'email...",../generatedResumes\4009701610.pdf,83.38
1,Machine Learning Engineer,https://www.linkedin.com/jobs/view/3992115810/...,KeyValue Software Systems,"Kochi, Kerala, India (On-site)",3992115810,About KeyValue\n\nKeyValue is a product develo...,63.32,"{'name': 'Muhammed Nishan', 'contact': {'email...",../generatedResumes\3992115810.pdf,66.15
2,Specialist - Machine Learning Engineer,https://www.linkedin.com/jobs/view/3992430870/...,LTIMindtree,"Bengaluru, Karnataka, India (Hybrid)",3992430870,Role Description\n\n3 – 6 years of relevant wo...,80.60,"{'name': 'Muhammed Nishan', 'contact': {'email...",../generatedResumes\3992430870.pdf,77.35
3,ML Engineer II,https://www.linkedin.com/jobs/view/4023099196/...,Nissan Motor Corporation,"Trivandrum, Kerala, India",4023099196,Nissan is a pioneer in Innovation and Technolo...,62.56,"{'name': 'Muhammed Nishan', 'contact': {'email...",../generatedResumes\4023099196.pdf,65.34
4,Machine Learning Engineer,https://www.linkedin.com/jobs/view/4016615439/...,VerSe Innovation,"Bengaluru, Karnataka, India (On-site)",4016615439,About the team\n\nAre you passionate about mac...,73.71,"{'name': 'Muhammed Nishan', 'contact': {'email...",../generatedResumes\4016615439.pdf,78.26
